In [1]:
import scala.io.Source
val sqlC = new org.apache.spark.sql.SQLContext(sc)
import sqlC.implicits._

In [2]:
case class Eval(sentences: String, pos: String, rdf: String, herd: String, predictions: String)
def evalIterator(filename: String): Iterator[Eval] = {
    for {
        line <- Source.fromFile(filename).getLines().drop(1)
        sentences = line.split("\t")(0)
        pos = line.split("\t")(1)
        rdf = line.split("\t")(2)
        herd = line.split("\t")(3)
        predictions = line.split("\t").drop(4).mkString(" ")
    } yield Eval(sentences, pos, rdf, herd, predictions)
}

In [23]:
val THRESHOLD = 0.75
def aboveThreshold(eval: Eval): Eval = {
    val probRe = """(0\.\d+)""".r
    
    eval.copy(predictions = eval.predictions.split("\t").filter(pred => {
        probRe.findFirstIn(pred) match {
            case Some(prob) => prob.toDouble >= THRESHOLD
            case _ => true // should never happen
        }
    }).mkString("\t"))
}

In [26]:
// test aboveThreshold
val eval = Eval("sentecs", "true", "rdf", "herd", predictions = "Q180589/P26/Q1900092 - 0.7921478929039968 - false\tQ180589/P26/Q4133024 - 0.8584705916062333 - false")
aboveThreshold(eval)

Eval(sentecs,true,rdf,herd,Q180589/P26/Q1900092 - 0.7921478929039968 - false	Q180589/P26/Q4133024 - 0.8584705916062333 - false)

# Place of Birth

In [53]:
val placeBirthFile = "2017-08-12_22.59.18-eval-place_of_birth_debug.tsv"
val placeBirth = evalIterator(placeBirthFile).map(aboveThreshold(_)).filter(_.pos == "true").toList
println(s"nbr of data points: ${placeBirth.size}")
val herdSuccessful = placeBirth.filter(_.herd == "true")
println(s"nbr herd successful: ${herdSuccessful.size}")
val herdIndependentTP = herdSuccessful.filter(_.predictions.split("\t").exists(_.contains("true")))
println(s"herd independent true positives: ${herdIndependentTP.size}")

val truePosOfHerdSuccessful = herdSuccessful.map(_.predictions.split("\t").map(p => if (p.contains("true")) 1 else 0).sum).sum                                                 
println(s"true positives where herd was succesful: $truePosOfHerdSuccessful")

nbr of data points: 1616
nbr herd successful: 756
herd independent true positives: 489
true positives where herd was succesful: 489


In [31]:
val placeBirthList = placeBirth.toList
placeBirthList
println(s"length of match: ${placeBirthList.length}")
val placeBirthDF = sc.parallelize(placeBirthList).toDF()

length of match: 505


In [32]:
%%dataframe
placeBirthDF

sentences,pos,rdf,herd,predictions
"Emmanuel Scheffer (, born 1 February 1924 in Germany) is an Israeli football coach.",true,Q2527091/P19/Q183,true,Q2527091/P19/Q183 - 0.7909471923941436 - true
"Juan Laporte (born November 24, 1959) is a former boxer who was born in Guayama, Puerto Rico.",true,Q1101615/P19/Q1870997,true,Q1101615/P19/Q1870997 - 0.8490445217839139 - true
"Vanessa Chinitor (born 13 October 1976, Dendermonde) is a Belgian singer, best known outside Belgium for her participation in the 1999 Eurovision Song Contest.",true,Q433783/P19/Q13122,true,Q433783/P19/Q13122 - 0.8984325541652719 - true
"Isaac Passy (; March 13, 1928, Plovdiv -- August 13, 2010) was a Bulgarian philosopher specializing in aesthetics. He was a professor at Sofia University from 1952 until 1993. He was the most prolific philosopher in the history of Bulgaria. He published over 40 monographs and over 80 volumes of the philosophical classics. He was the father of Solomon Passy.",true,Q2901020/P19/Q459,true,Q2901020/P19/Q459 - 0.7995216933144685 - true
"Francisco de Osuna (1497--1541) was a Spanish author, born in Seville.",true,Q921537/P19/Q8717,true,Q921537/P19/Q8717 - 0.932614625756665 - true
"Lars Gabriel von Haartman (23 September 1789 Turku - 16 December 1859 Merimasku) was a Swedish-speaking Finnish politician. Along with Count Gustaf Mauritz Armfelt, he was one of the most prominent politicians who were in favour of developing Finland as part of the Russian Empire instead of striving towards independence.",true,Q2564079/P19/Q38511,true,Q2564079/P19/Q38511 - 0.7705465541778971 - true
"Nina Repeta (born September 10, 1967 in Shelby, North Carolina) is an American actress best known for her role as Bessie Potter on the television drama Dawson's Creek, which aired from 1998 to 2003.",true,Q177110/P19/Q2029222,true,Q177110/P19/Q2029222 - 0.8620242418814815 - true
"Mark Koevermans (born 3 February 1968 in Rotterdam) is a former tennis player from the Netherlands, who turned professional in 1987. He represented his native country as a lucky loser at the 1992 Summer Olympics in Barcelona, where he was defeated in the third round by Brazil's Jaime Oncins. The right-hander, nicknamed Koef, won one career title in singles (Athens, 1990). He reached his highest singles ATP-ranking on 27 May 1991, when he became the number 37 of the world.",true,Q610060/P19/Q34370,true,Q610060/P19/Q1524 - 0.7813850128585691 - false Q610060/P19/Q34370 - 0.8822166471274631 - true
"Małgorzata Olejnik (born June 3, 1966 in Kielce) is a Polish politician. She was elected to Sejm on September 25, 2005 getting 12398 votes in 33 Kielce district, candidating from Samoobrona Rzeczpospolitej Polskiej list.",true,Q2752384/P19/Q102317,true,Q2752384/P19/Q102317 - 0.8667805576273522 - true
"Rick Kelly (born on 17 January 1983 in Mildura, Victoria, Australia) is a current V8 Supercar racing driver, currently living in Melbourne, Victoria. He is the younger brother of former Bathurst 1000 champion, Todd Kelly. He previously drove for the HSV Dealer Team and has previously contributed to winning at the Bathurst 1000 alongside Greg Murphy in 2003 and 2004 under the banner of Kmart Racing. Kelly was the 2006 V8 Supercar Champion for the HSV Dealer Team. He moved to the new formed family team Kelly Racing for 2009 and onwards.",true,Q2742848/P19/Q705829,true,Q2742848/P19/Q36687 - 0.8328804537474631 - false Q2742848/P19/Q705829 - 0.8721254725037106 - true


* Q3388789 = Morris S Miller
* Q11701 = US House of Repr.
* Q60 = NYC
* Q1567748 = Union College
* Q331380 = Schenectady

# Institution

In [54]:
val instFile = "2017-08-12_22.59.18-eval_institution_debug.tsv"
val inst = evalIterator(instFile).map(aboveThreshold(_)).filter(_.pos == "true").toList

println(s"nbr of data points: ${inst.size}")
val herdSuccessful = inst.filter(_.herd == "true")
println(s"nbr herd successful: ${herdSuccessful.size}")

val truePosOfHerdSuccessful = herdSuccessful.map(_.predictions.split("\t").map(p => if (p.contains("true")) 1 else 0).sum).sum                                                 
val herdIndependentTP = herdSuccessful.filter(_.predictions.split("\t").exists(_.contains("true")))
println(s"herd independent true positives: ${herdIndependentTP.size}")
println(s"true positives where herd was succesful: $truePosOfHerdSuccessful")

nbr of data points: 3850
nbr herd successful: 791
herd independent true positives: 71
true positives where herd was succesful: 71


In [57]:
val instList = herdSuccessful.filter(_.predictions.split("\t").exists(_.contains("false"))).toList
instList
println(s"length of match where herd was sucessful but we were not: ${instList.length}")
val instDF = sc.parallelize(instList).toDF()

length of match where herd was sucessful but we were not: 239


In [56]:
%%dataframe
instDF

sentences,pos,rdf,herd,predictions
"John Courtney Murray was born in New York City in 1904 and entered the New York province of the Society of Jesus in 1920. He studied Classics and Philosophy at Boston College, receiving bachelor's and master's degrees in 1926 and 1927 respectively. Following his graduation, he travelled to the Philippines where he taught Latin and English literature at the Ateneo de Manila. He returned to the United States in 1930 and was ordained a Roman Catholic priest in 1933. He pursued further studies at the Gregorian University in Rome and completed a doctorate in sacred theology in 1937. Returning to the United States, he taught Catholic trinitarian theology at the Jesuit theologate at Woodstock, Maryland and, in 1941, was named editor of the Jesuit journal Theological Studies. He held both positions until his death in Queens, New York in 1967. He died of a heart attack.",true,Q2619214/P69/Q467025,true,Q2619214/P19/Q60 - 0.8121855113438503 - false
"Born and raised in New York City, novelist, poet and performance artist Kathy Acker came to be closely associated with the punk movement of the 1970s and '80s that affected much of the culture in and around Manhattan. As an adult, however, she moved around quite a bit. She received her bachelor's degree from the University of California, San Diego in 1968; there she worked with David Antin and Jerome Rothenberg.",true,Q270641/P69/Q622664,true,Q270641/P19/Q11299 - 0.7647229653322255 - false Q5230808/P26/Q1775056 - 0.6636213926570664 - false
"Mirrione was born in Santa Clara County, California. He attended Bellarmine College Preparatory and then the University of California, Santa Cruz, from which he received his bachelor's degree in 1991. He moved to Los Angeles, and began a collaboration with Doug Liman, who was then a graduate student at the University of Southern California School of Cinematic Arts. Mirrione edited Liman's first feature films Getting In (1994), Swingers (1996), and Go (1999), which was an homage to Akira Kurosawa's 1950 film Rashomon.",true,Q349339/P69/Q1583249,true,Q349339/P26/Q8006 - 0.9147220426094974 - false Q349339/P69/Q1583249 - 0.7300737065804097 - true Q8006/P26/Q349339 - 0.8328457211464527 - false
"Ildebrando Antoniutti was born in Nimis, the sixth of the seven children of Giuseppe and Anna (née Comelli) Antoniutti. Studying at the seminaries in Cividale and Udine, he accompanied Archbishop Antonio Anastasio Rossi of Udine during World War I on Rossi's frequent visits to military hospitals and to parishes in areas occupied by Austro-Hungarian forces. In November 1917 Antoniutti went to Rome, where he studied at the Pontifical Roman Seminary and Pontifical Lateran University (from where he obtained his doctorates in philosophy, theology, and canon law in July 1920).",true,Q1114130/P69/Q475007,true,Q1114130/P19/Q53296 - 0.8303944602910945 - false
"Ivan Lins was born in Rio de Janeiro. He spent several years in Boston, Massachusetts, while his father, a naval engineer, continued his graduate studies at M.I.T., studied at the Military College in Rio. He later received a degree in industrial chemical engineering from the Federal University of Rio de Janeiro. He considered a career in volleyball before discovering his considerable musical talent. Ivan Lins currently resides in Rio de Janeiro, Brazil.",true,Q703330/P69/Q586904,true,Q703330/P19/Q8678 - 0.86262253462012 - false
"Anton Buttigieg was born in Qala, Gozo, on February 19, 1912, the third child of Salvatore and Concetta (née Falzon). He was educated at the Government Primary School, Qala (1916--1922), the Gozo Seminary (1923--1927), St Aloysius' College Malta (1928--1930) and the University of Malta, where he graduated Bachelor of Arts in 1934 and Doctor of Laws in 1940.",true,Q250177/P69/Q426045,true,Q250177/P19/Q1088033 - 0.8432996120783868 - false Q1088033/P69/Q426045 - 0.5894338874918948 - false Q250177/P19/Q170488 - 0.700372185176385 - false
"Karl Eduard von Holtei was bor

Some of the errors is due to coreferences. So HERD has succeeded in extracting the NED:s but not for the sentence where the institution is stated.

Also note that there are a lot of extractions marked false that are not necessarily false; they are extracted for a different relation - place of birth when the evaluation only looks at institution.
This could possibly be remedied by adding filtering out 